In [62]:
import pandas as pd
import tiktoken


In [63]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [64]:
# load & inspect dataset
import pandas as pd

input_datapath = "test_embeddings_major-andminor-yes-cosine-simlarity.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath)

In [65]:

# Selecting specific columns
selected_df = df[['Concatenated_SASB_Query', 'title_and_content']]



In [67]:
pd.set_option('display.max_colwidth', 50)
selected_df

,Concatenated_SASB_Query,title_and_content
0,Tailings Storage Facilities Management - The M...,New York Cements Itself as the Gold Mining Cap...
1,Tailings Storage Facilities Management - The M...,Sagging Chinese Growth Won‚Äö√Ñ√¥t Punish Copp...
2,Tailings Storage Facilities Management - The M...,Barrick Gold open to acquiring Newmont's share...
3,Tailings Storage Facilities Management - The M...,Merger activity among gold producers suggests ...
4,Tailings Storage Facilities Management - The M...,The amount of gas sent to U.S. LNG export term...
...,...,...
1761,"Land Use & Ecological Impacts - Meat, Poultry ...",Spring 2023 Layoff Tracker: Tyson Foods Cuts O...
1762,"Land Use & Ecological Impacts - Meat, Poultry ...",Tyson enlists self-driving trucks to deliver c...
1763,"Land Use & Ecological Impacts - Meat, Poultry ...",Tyson enlists self-driving trucks to deliver c...
1764,"Land Use & Ecological Impacts - Meat, Poultry ...",Big Meat Companies Want to Use Smartwatches to...


In [72]:
import pandas as pd
import openai
from openai import OpenAI

# Initialize OpenAI client with your API key
client = OpenAI(api_key="your_api_key_here")

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")  # Clean up new line characters
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding


In [73]:
# Create empty lists to store embeddings
ada_embeddings_SASB_Query = []
ada_embeddings_SASB_Candidate = []

# Loop through each row in the DataFrame
for index, row in selected_df.iterrows():
    # Compute embeddings for each column
    query_embedding = get_embedding(row['Concatenated_SASB_Query'], model='text-embedding-3-small')
    candidate_embedding = get_embedding(row['title_and_content'], model='text-embedding-3-small')
    
    # Append embeddings to their respective lists
    ada_embeddings_SASB_Query.append(query_embedding)
    ada_embeddings_SASB_Candidate.append(candidate_embedding)

# Assign computed embeddings back to the DataFrame
selected_df['ada_embedding_SASB_Query'] = ada_embeddings_SASB_Query
selected_df['ada_embedding_SASB_Candidate'] = ada_embeddings_SASB_Candidate

/var/folders/m0/76k8l35x2p15kn84b0cplr8c0000gn/T/ipykernel_75029/2930129265.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['ada_embedding_SASB_Query'] = ada_embeddings_SASB_Query
/var/folders/m0/76k8l35x2p15kn84b0cplr8c0000gn/T/ipykernel_75029/2930129265.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['ada_embedding_SASB_Candidate'] = ada_embeddings_SASB_Candidate


In [76]:
selected_df

,Concatenated_SASB_Query,title_and_content,ada_embedding_SASB_Query,ada_embedding_SASB_Candidate
0,Tailings Storage Facilities Management - The M...,New York Cements Itself as the Gold Mining Cap...,"[0.017493782564997673, 0.051179543137550354, 0...","[0.0015000662533566356, -0.003786575049161911,..."
1,Tailings Storage Facilities Management - The M...,Sagging Chinese Growth Won‚Äö√Ñ√¥t Punish Copp...,"[0.017489079385995865, 0.051165781915187836, 0...","[0.014376699924468994, -0.032034337520599365, ..."


In [77]:
df_final = selected_df[['Concatenated_SASB_Query', 'title_and_content', 'ada_embedding_SASB_Query', 'ada_embedding_SASB_Candidate']]

In [78]:
df = df[['query_embedding', 'candidate_embedding']]

In [80]:
data_compare = pd.concat([df_final, df], axis=1)

In [81]:
data_compare.head(1)

,Concatenated_SASB_Query,title_and_content,ada_embedding_SASB_Query,ada_embedding_SASB_Candidate,query_embedding,candidate_embedding
0,Tailings Storage Facilities Management - The M...,New York Cements Itself as the Gold Mining Cap...,"[0.017493782564997673, 0.051179543137550354, 0...","[0.0015000662533566356, -0.003786575049161911,...",[[0.9865462 0.01918043 0.00753147 0.11734246 ...,[[0.52431774 0.4818942 0.5384123 0.5376571 ...


In [83]:
# Get the length of the list in the first row of the 'ada_embedding_SASB_Query' column
l1 = len(data_compare['ada_embedding_SASB_Query'].iloc[0])
l2 = len(data_compare['ada_embedding_SASB_Candidate'].iloc[0])
l3 = len(data_compare['query_embedding'].iloc[0])
l4 = len(data_compare['candidate_embedding'].iloc[0])

# Print the result
print("Length of the ADA Query list:", l1)
print("Length of the ADA Candidate list:", l2)
print("Length of the Two Tower Query list:", l3)
print("Length of the Two Tower Candidate list:", l4)

Length of the ADA Query list: 1536
Length of the ADA Candidate list: 1536
Length of the Two Tower Query list: 58
Length of the Two Tower Candidate list: 58


In [82]:
data_compare.to_csv('ADA-test-final.csv')  # Optionally, save to a new CSV file